In [407]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [408]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
# for windows
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Reading text from .txt

In [409]:
wizard_of_oz_book = "wizard_of_oz.txt"
shakespeare = "shakespeare.txt"

cars = ""
with open(shakespeare, 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

## Tokenizer

In [ ]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
data[:100] # First 100 characters of book tokonized

In [411]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

## Getting batch from text

In [412]:
batch_size = 32
block_size = 128

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([32, 128])
tensor([[52, 56, 63,  ..., 56, 54, 43],
        [45, 43, 56,  ..., 39, 58,  1],
        [ 6,  1, 39,  ..., 47, 57,  1],
        ...,
        [46, 53, 57,  ..., 33, 17, 17],
        [32, 20, 10,  ...,  0, 37, 53],
        [43,  1, 58,  ..., 61,  1, 57]], device='mps:0')
targets:
torch.Size([32, 128])
tensor([[56, 63,  5,  ..., 54, 43, 56],
        [43, 56,  1,  ..., 58,  1, 52],
        [ 1, 39, 52,  ..., 57,  1, 45],
        ...,
        [53, 57, 58,  ..., 17, 17, 26],
        [20, 10,  0,  ..., 37, 53, 59],
        [ 1, 58, 46,  ...,  1, 57, 53]], device='mps:0')


## Model paramerters

In [413]:
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

# LLM Model

In [414]:
class Head(nn.Module):
    # one head of self-attention
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [415]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [416]:
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [417]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [418]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    # Initialize weights and bias
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):        
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:                        
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            
            # get the predictions
            logits, loss = self.forward(index_cond)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1) 
            
        return index

In [419]:
m = GPTLanguageModel(vocab_size)
model = m.to(device)

# Optimizer + Training

In [431]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
steps = 3000

In [432]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(250)
        for k in range(250):
            X, Y = get_batch(split)
            logits, loss = model.forward(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [433]:
for step in range(steps):
    if step % 250 == 0:
        losses = estimate_loss()
        print(f"step: {step}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 1.285, val loss: 1.529
step: 250, train loss: 1.278, val loss: 1.527
step: 500, train loss: 1.248, val loss: 1.519
step: 750, train loss: 1.239, val loss: 1.492
step: 1000, train loss: 1.220, val loss: 1.496
step: 1250, train loss: 1.204, val loss: 1.499
step: 1500, train loss: 1.193, val loss: 1.496
step: 1750, train loss: 1.183, val loss: 1.489
step: 2000, train loss: 1.173, val loss: 1.485
step: 2250, train loss: 1.156, val loss: 1.486
step: 2500, train loss: 1.143, val loss: 1.498
step: 2750, train loss: 1.132, val loss: 1.489
1.2248213291168213


In [434]:
prompt = "We are accounted poor citizens"
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=500)[0].tolist())
print(generated_chars)

We are accounted poor citizens,
Were yet scorn, we wept your highness.

PARIS:
Good deposed,
And I have, by old Cupitol, what shame you fnurst
Could not fourten.

CAMILLO:
Are you king?

HENRY PERCZABETH:
Commands, we'll away to content, all did to fight;
And will you colour please. Plantage your honours;
And procloxe pranks, are you your ward cozen'd:
For as tymall shall be gone, and we know them chosel.
We have more heaven no oppointable of his riseon,
With confince to know you, nobler,
Warwick, peace; and changing them fa
